In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re
import pickle
from tqdm import tqdm
from constants import base_path, max_n_questions
import matplotlib.pyplot as plt
from collections import Counter
tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
tqdm.pandas()

In [2]:
with open(f"{base_path}\\input\\release_conditions.json") as f:
  disease_dict = json.load(f)
disease_list = list(disease_dict.keys())

In [3]:
with open(f"{base_path}\\input\\release_evidences.json") as f:
  evidences = json.load(f)
evidences_list = []
evidences_dict = {}
evidences_en_to_code = {}
for e in evidences.keys():
  # only binary symptoms and no antecedents
  if (not evidences[e]["possible-values"]) and (not evidences[e]["is_antecedent"]):
    evidences_list.append(e)
    evidences_dict[e] = evidences[e]["question_en"]
    evidences_en_to_code[evidences[e]["question_en"]] = e
evidences_code_to_en = evidences_dict
evidences_list_en = list(evidences_en_to_code.keys())
evidences_dict["AGE"] = "AGE"
evidences_dict["SEX"] = "SEX"
feature_columns = ["AGE", "SEX"] + evidences_list

In [4]:
def get_next_question(evidences, questionnaire, feature_embeddings_df):
    centroid = np.array([feature_embeddings_df.loc[e].values for e in evidences]).mean(axis=0)
    _, indices = questionnaire.kneighbors([centroid])
    ask_list = [evidences_list_en[i] for i in indices[0] if evidences_list_en[i] not in evidences]
    try:
        return ask_list
    except:
        return []

In [5]:
def get_evidences(answers, user_evidences, questionnaire, feature_embeddings_df):
    ask = True
    question_counter = 1 # counts initial evidence sa q0
    evidences_en = [evidences_code_to_en[e] for e in user_evidences]
    answers_en = [evidences_code_to_en[e] for e in answers if e in evidences_list]
    asked = evidences_en.copy()
    while question_counter < max_n_questions:
      ask =  True
      next_question_idx = 0
      next_question = get_next_question(evidences_en, questionnaire, feature_embeddings_df)
      while ask and (question_counter < max_n_questions):
        if next_question[next_question_idx] not in asked:
          answer = 1 if next_question[next_question_idx] in answers_en else 0
          asked.append(next_question[next_question_idx])
          question_counter+=1
          if answer==1:
            evidences_en.append(next_question[next_question_idx])
            ask = False
          else:
            next_question_idx += 1
          if next_question_idx > max_n_questions:
            break
        else:
            next_question_idx += 1
    return [evidences_en_to_code[e] for e in evidences_en]

In [6]:
def data_proc(df, questionnaire, feature_embeddings_df):
    df["binary_evidences_all"] = df["EVIDENCES"].apply(lambda x: [d for d in ast.literal_eval(x) if d in evidences_list])
    df["binary_evidences_all_count"] = df["binary_evidences_all"].apply(lambda x: len(x))
    df["binary_evidences"] = df[["EVIDENCES", "INITIAL_EVIDENCE"]].progress_apply(lambda x: get_evidences(ast.literal_eval(x[0]), [x[1]], questionnaire, feature_embeddings_df), axis=1)
    df["binary_evidences_count"] = df["binary_evidences"].apply(lambda x: len(x))
    df["hit_rate"] = df["binary_evidences_count"]/df["binary_evidences_all_count"]
    hit_rate = df["hit_rate"]
    for e in evidences_list:
        df[e] = df["binary_evidences"].apply(lambda x: 1 if e in x else 0)
    df["SEX"] = df["SEX"].map({'F': 0, 'M': 1})
    ftr_df = df[feature_columns + ["PATHOLOGY"]]
    questionnaire_df = df[["binary_evidences_all", "binary_evidences", "INITIAL_EVIDENCE"]]
    return ftr_df, hit_rate, questionnaire_df

In [7]:
def pred(x):
    pred_list = []
    for i in range(len(disease_list)):
        if x[i] > 0:
            pred_list.append({
                "disease": disease_list[i],
                "probability": x[i]})
    if pred_list:
        pred_df = pd.DataFrame(pred_list).set_index('disease')
        # return only top 1 - allows ties
        pred_df['rank'] = pred_df['probability'].rank(method='min', ascending=False)
        pred_df = pred_df.sort_values(by="rank")
        pred_df = pred_df[pred_df["rank"]<=1][["probability"]]
        return list(pred_df.index)
    else:
        return []

In [8]:
def get_missed_evidence(actual, asked):
    return list(set(actual)^set(asked))

In [9]:
def validate(x):
    return [x[0]]==x[1]

In [10]:
with open(f'{base_path}\\output\\questionnaire\\questionnaire.pkl', 'rb') as f:
    questionnaire = pickle.load(f)
feature_embeddings_df = pd.read_pickle(f'{base_path}\\output\\questionnaire\\questionnaire_embeddings.pkl')

In [11]:
diagnosis_df_valid = pd.read_csv(f"{base_path}\\input\\release_validate_patients")

## Random Forest

In [12]:
model_dict = {}
for disease in disease_list:
    disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
    with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{disease_filename}_model.pkl', 'rb') as f:
        model_dict[disease] = pickle.load(f)

In [13]:
data_proc(diagnosis_df_valid.head(5), questionnaire, feature_embeddings_df)

100%|██████████| 5/5 [00:00<00:00, 134.03it/s]


(   AGE  SEX  E_91  E_53  E_159  E_129  E_154  E_155  E_210  E_140  ...  E_193  \
 0   55    0     0     1      0      0      1      0      0      1  ...      0   
 1   10    0     0     1      0      0      0      1      0      0  ...      0   
 2   68    0     1     1      0      1      0      0      0      0  ...      0   
 3   13    1     0     1      0      0      0      0      0      0  ...      0   
 4   48    1     0     1      0      0      0      0      1      0  ...      0   
 
    E_168  E_180  E_67  E_171  E_111  E_182  E_103  E_23     PATHOLOGY  
 0      0      0     0      0      0      0      0     0        Anemia  
 1      0      0     0      1      1      0      0     0  Panic attack  
 2      0      0     0      0      0      0      0     0     Influenza  
 3      0      0     0      0      0      0      0     0        Anemia  
 4      0      0     0      0      0      0      0     0     Boerhaave  
 
 [5 rows x 99 columns],
 0    1.0
 1    1.0
 2    1.0
 3    1.0
 4

In [14]:
diagnosis_df_valid, hit_rate, questionnaire_df = data_proc(diagnosis_df_valid, questionnaire, feature_embeddings_df)
# sample x% of the validation dataset
# diagnosis_df_valid = diagnosis_df_valid.sample(frac=0.01, random_state=1)
diagnosis_df_valid

100%|██████████| 132448/132448 [10:57<00:00, 201.56it/s]


,AGE,SEX,E_91,E_53,E_159,E_129,E_154,E_155,E_210,E_140,...,E_193,E_168,E_180,E_67,E_171,E_111,E_182,E_103,E_23,PATHOLOGY
0,55,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,Anemia
1,10,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,Panic attack
2,68,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Influenza
3,13,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Anemia
4,48,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Boerhaave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132443,27,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Viral pharyngitis
132444,57,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,Acute pulmonary edema
132445,52,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GERD
132446,10,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Epiglottitis


In [15]:
questionnaire_df["missed_evidence"] = questionnaire_df.progress_apply(lambda x: get_missed_evidence(x[0], x[1]), axis=1)
questionnaire_df

100%|██████████| 132448/132448 [00:03<00:00, 39335.18it/s]


,binary_evidences_all,binary_evidences,INITIAL_EVIDENCE,missed_evidence
0,"[E_53, E_76, E_82, E_88, E_89, E_140, E_154]","[E_154, E_140, E_88, E_76, E_82, E_53, E_89]",E_154,[]
1,"[E_50, E_53, E_66, E_75, E_111, E_155, E_157, ...","[E_171, E_111, E_75, E_157, E_177, E_155, E_50...",E_171,[]
2,"[E_50, E_53, E_88, E_91, E_94, E_129, E_144, E...","[E_53, E_91, E_181, E_144, E_50, E_129, E_94, ...",E_53,[]
3,"[E_53, E_66, E_76, E_82, E_89, E_179]","[E_53, E_66, E_89, E_82, E_76, E_179]",E_53,[]
4,"[E_53, E_148, E_210]","[E_53, E_148, E_210]",E_53,[]
...,...,...,...,...
132443,"[E_53, E_181, E_201]","[E_201, E_181, E_53]",E_201,[]
132444,"[E_53, E_66, E_67, E_151, E_217]","[E_151, E_67, E_217, E_53, E_66]",E_151,[]
132445,"[E_53, E_173, E_201, E_215]","[E_173, E_215, E_201, E_53]",E_173,[]
132446,"[E_53, E_65, E_91, E_190, E_194]","[E_91, E_53, E_190, E_194]",E_91,[E_65]


In [16]:
questionnaire_df.to_csv(f"{base_path}\\output\\error_analysis_questionnaire\\questionnaire_df.csv")

In [17]:
missed_evidences = []
for e in questionnaire_df["missed_evidence"]:
    if e:
        missed_evidences.extend(e)

In [18]:
missed_evidences_dict = dict(Counter(missed_evidences).most_common(10))
missed_evidences_dict = {evidences_code_to_en[k]:missed_evidences_dict[k] for k in missed_evidences_dict}
missed_evidences_dict

{'Have you noticed a high pitched sound when breathing in?': 983,
 'Do you feel that your eyes produce excessive tears?': 666,
 'Do you have difficulty swallowing, or have a feeling of discomfort/blockage when swallowing?': 582,
 'Have you recently thrown up blood or something resembling coffee beans?': 423,
 'Do you have pain that improves when you lean forward?': 413,
 'Are you more irritable or has your mood been very unstable recently?': 394,
 'Do you have a burning sensation that starts in your stomach then goes up into your throat, and can be associated with a bitter taste in your mouth?': 392,
 'Do you have symptoms that get worse after eating?': 385,
 'Do you feel slightly dizzy or lightheaded?': 330,
 'Have you lost consciousness associated with violent and sustained muscle contractions or had an absence episode?': 305}

In [19]:
hit_rate.plot.hist()
plt.ylabel("Frequency")
plt.title("Hit rate Distribution")
plt.savefig(f'{base_path}\\output\\questionnaire\\hit_rate.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [20]:
counter = 0
for disease in disease_list:
    rf_model = model_dict[disease]
    diagnosis_df_valid[disease] = rf_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
    counter+=1
    print(f"done {counter}: {disease}")

done 1: Spontaneous pneumothorax
done 2: Cluster headache
done 3: Boerhaave
done 4: Spontaneous rib fracture
done 5: GERD
done 6: HIV (initial infection)
done 7: Anemia
done 8: Viral pharyngitis
done 9: Inguinal hernia
done 10: Myasthenia gravis
done 11: Whooping cough
done 12: Anaphylaxis
done 13: Epiglottitis
done 14: Guillain-Barré syndrome
done 15: Acute laryngitis
done 16: Croup
done 17: PSVT
done 18: Atrial fibrillation
done 19: Bronchiectasis
done 20: Allergic sinusitis
done 21: Chagas
done 22: Scombroid food poisoning
done 23: Myocarditis
done 24: Larygospasm
done 25: Acute dystonic reactions
done 26: Localized edema
done 27: SLE
done 28: Tuberculosis
done 29: Unstable angina
done 30: Stable angina
done 31: Ebola
done 32: Acute otitis media
done 33: Panic attack
done 34: Bronchospasm / acute asthma exacerbation
done 35: Bronchitis
done 36: Acute COPD exacerbation / infection
done 37: Pulmonary embolism
done 38: URTI
done 39: Influenza
done 40: Pneumonia
done 41: Acute rhinosinu

In [21]:
diagnosis_df_valid

,AGE,SEX,E_91,E_53,E_159,E_129,E_154,E_155,E_210,E_140,...,Pneumonia,Acute rhinosinusitis,Chronic rhinosinusitis,Bronchiolitis,Pulmonary neoplasm,Possible NSTEMI / STEMI,Sarcoidosis,Pancreatic neoplasm,Acute pulmonary edema,Pericarditis
0,55,0,0,1,0,0,1,0,0,1,...,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000
1,10,0,0,1,0,0,0,1,0,0,...,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.003
2,68,0,1,1,0,1,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000
3,13,1,0,1,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000
4,48,1,0,1,0,0,0,0,1,0,...,0.0,0.000000,0.000000,0.0,0.00,0.09,0.00,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132443,27,1,0,1,0,0,0,0,0,0,...,0.0,0.414463,0.702787,0.0,0.00,0.00,0.00,0.0,0.0,0.000
132444,57,1,0,1,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.01,0.00,0.00,0.0,1.0,0.010
132445,52,0,0,1,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000
132446,10,1,1,1,0,0,0,0,0,0,...,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000


In [22]:
diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)

  0%|          | 0/132448 [00:00<?, ?it/s]

100%|██████████| 132448/132448 [06:16<00:00, 351.43it/s]


In [23]:
diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)

  3%|▎         | 3618/132448 [00:00<00:03, 35837.58it/s]

100%|██████████| 132448/132448 [00:03<00:00, 42832.71it/s]


In [24]:
diagnosis_df_valid

,AGE,SEX,E_91,E_53,E_159,E_129,E_154,E_155,E_210,E_140,...,Chronic rhinosinusitis,Bronchiolitis,Pulmonary neoplasm,Possible NSTEMI / STEMI,Sarcoidosis,Pancreatic neoplasm,Acute pulmonary edema,Pericarditis,predicted_diagnosis,is_matched
0,55,0,0,1,0,0,1,0,0,1,...,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000,[Anemia],True
1,10,0,0,1,0,0,0,1,0,0,...,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.003,[Panic attack],True
2,68,0,1,1,0,1,0,0,0,0,...,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000,[Influenza],True
3,13,1,0,1,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000,[Anemia],True
4,48,1,0,1,0,0,0,0,1,0,...,0.000000,0.0,0.00,0.09,0.00,0.0,0.0,0.000,[Boerhaave],True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132443,27,1,0,1,0,0,0,0,0,0,...,0.702787,0.0,0.00,0.00,0.00,0.0,0.0,0.000,[Acute otitis media],False
132444,57,1,0,1,0,0,0,0,0,0,...,0.000000,0.0,0.01,0.00,0.00,0.0,1.0,0.010,[Acute pulmonary edema],True
132445,52,0,0,1,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000,[GERD],True
132446,10,1,1,1,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.000,[Epiglottitis],True


In [25]:
diagnosis_df_valid["hit_rate"] = hit_rate

In [26]:
diagnosis_df_valid.is_matched.value_counts()

True     117269
False     15179
Name: is_matched, dtype: int64

In [27]:
acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
acc

{True: 117269, False: 15179, 'error_rate': 0.11460346702101958}

In [28]:
np.mean(hit_rate)

0.9840657983621638

In [29]:
acc["mean_hit_rate"] = np.mean(hit_rate)

In [30]:
# hit rate of misdiagnosed
hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
hit_rate_misdiagnosed.plot.hist()
plt.ylabel("Frequency")
plt.title("Hit rate Distribution - Misdiagnosed")
plt.savefig(f'{base_path}\\output\\questionnaire\\hit_rate_misdiagnosed.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [31]:
acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)

In [32]:
with open(f"{base_path}\\output\\error_analysis_questionnaire\\validation_metric_questionnaire.json", "w") as outfile: 
    json.dump(acc, outfile, indent=True)

In [33]:
diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis", "is_matched", "hit_rate"] + disease_list].to_csv(f"{base_path}\\output\\error_analysis_questionnaire\\validation_df_all_patients_questionnaire.csv")

## Logistic Regression

In [34]:
model_dict = {}
for disease in disease_list:
    disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
    with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{disease_filename}_logreg_model.pkl', 'rb') as f:
        model_dict[disease] = pickle.load(f)

In [35]:
counter = 0
for disease in disease_list:
    logreg_model = model_dict[disease]
    diagnosis_df_valid[disease] = logreg_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
    counter+=1
    print(f"done {counter}: {disease}")

done 1: Spontaneous pneumothorax
done 2: Cluster headache
done 3: Boerhaave
done 4: Spontaneous rib fracture
done 5: GERD
done 6: HIV (initial infection)
done 7: Anemia
done 8: Viral pharyngitis
done 9: Inguinal hernia
done 10: Myasthenia gravis
done 11: Whooping cough
done 12: Anaphylaxis
done 13: Epiglottitis
done 14: Guillain-Barré syndrome
done 15: Acute laryngitis
done 16: Croup
done 17: PSVT
done 18: Atrial fibrillation
done 19: Bronchiectasis
done 20: Allergic sinusitis
done 21: Chagas
done 22: Scombroid food poisoning
done 23: Myocarditis
done 24: Larygospasm
done 25: Acute dystonic reactions
done 26: Localized edema
done 27: SLE
done 28: Tuberculosis
done 29: Unstable angina
done 30: Stable angina
done 31: Ebola
done 32: Acute otitis media
done 33: Panic attack
done 34: Bronchospasm / acute asthma exacerbation
done 35: Bronchitis
done 36: Acute COPD exacerbation / infection
done 37: Pulmonary embolism
done 38: URTI
done 39: Influenza
done 40: Pneumonia
done 41: Acute rhinosinu

In [36]:
diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)

  0%|          | 0/132448 [00:00<?, ?it/s]

100%|██████████| 132448/132448 [06:08<00:00, 359.04it/s]


In [37]:
diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)

  0%|          | 0/132448 [00:00<?, ?it/s]

100%|██████████| 132448/132448 [00:03<00:00, 41501.84it/s]


In [38]:
diagnosis_df_valid

,AGE,SEX,E_91,E_53,E_159,E_129,E_154,E_155,E_210,E_140,...,Bronchiolitis,Pulmonary neoplasm,Possible NSTEMI / STEMI,Sarcoidosis,Pancreatic neoplasm,Acute pulmonary edema,Pericarditis,predicted_diagnosis,is_matched,hit_rate
0,55,0,0,1,0,0,1,0,0,1,...,4.156345e-25,9.971866e-06,5.167661e-07,1.204016e-05,1.259412e-05,1.749385e-07,2.262037e-11,[Anemia],True,1.0
1,10,0,0,1,0,0,0,1,0,0,...,9.456161e-06,7.512104e-07,1.435395e-06,1.222314e-04,7.887219e-08,9.039293e-05,2.043855e-05,[Panic attack],True,1.0
2,68,0,1,1,0,1,0,0,0,0,...,1.120161e-29,4.021024e-11,1.587541e-06,4.225942e-09,2.795075e-11,1.416207e-06,1.072775e-10,[Influenza],True,1.0
3,13,1,0,1,0,0,0,0,0,0,...,1.370537e-06,9.743676e-04,1.436016e-04,4.686427e-04,5.978126e-06,1.461622e-05,1.750324e-08,[Anemia],True,1.0
4,48,1,0,1,0,0,0,0,1,0,...,5.213556e-22,2.279438e-06,4.176768e-02,1.158820e-04,2.148152e-02,4.861522e-07,1.714267e-04,[Boerhaave],True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132443,27,1,0,1,0,0,0,0,0,0,...,6.266019e-12,2.593481e-04,1.940074e-05,3.897018e-04,4.993563e-03,1.110333e-06,3.782573e-05,[Viral pharyngitis],True,1.0
132444,57,1,0,1,0,0,0,0,0,0,...,3.020683e-25,9.024875e-04,1.140563e-04,1.114355e-03,2.156294e-07,9.999477e-01,7.069921e-02,[Acute pulmonary edema],True,1.0
132445,52,0,0,1,0,0,0,0,0,0,...,1.595498e-23,8.437705e-03,5.051341e-05,4.315803e-04,6.298523e-03,1.031900e-06,1.934437e-07,[GERD],True,1.0
132446,10,1,1,1,0,0,0,0,0,0,...,3.171149e-05,5.816555e-07,3.190883e-04,8.660012e-05,1.952470e-05,3.389337e-07,6.529514e-04,[Epiglottitis],True,0.8


In [39]:
diagnosis_df_valid["hit_rate"] = hit_rate

In [40]:
diagnosis_df_valid.is_matched.value_counts()

True     122498
False      9950
Name: is_matched, dtype: int64

In [41]:
acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
acc

{True: 122498, False: 9950, 'error_rate': 0.07512382217927036}

In [42]:
np.mean(hit_rate)

0.9840657983621638

In [43]:
acc["mean_hit_rate"] = np.mean(hit_rate)

In [44]:
# hit rate of misdiagnosed
hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
hit_rate_misdiagnosed.plot.hist()
plt.ylabel("Frequency")
plt.title("Hit rate Distribution - Misdiagnosed")
plt.savefig(f'{base_path}\\output\\questionnaire\\hit_rate_misdiagnosed_logreg.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [45]:
np.mean(hit_rate_misdiagnosed)

0.87413496051687

In [46]:
acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)

In [47]:
with open(f"{base_path}\\output\\error_analysis_questionnaire\\validation_metric_logreg_questionnaire.json", "w") as outfile: 
    json.dump(acc, outfile, indent=True)

In [48]:
diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis", "is_matched", "hit_rate"] + disease_list].to_csv(f"{base_path}\\output\\error_analysis_questionnaire\\validation_logreg_df_all_patients_questionnaire.csv")

## Decision Tree

In [49]:
model_dict = {}
for disease in disease_list:
    disease_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
    with open(f'{base_path}\\output\\diseases\\{disease_filename}\\{disease_filename}_dt_model.pkl', 'rb') as f:
        model_dict[disease] = pickle.load(f)

In [50]:
counter = 0
for disease in disease_list:
    logreg_model = model_dict[disease]
    diagnosis_df_valid[disease] = logreg_model.predict_proba(diagnosis_df_valid[feature_columns])[:,1]
    counter+=1
    print(f"done {counter}: {disease}")

done 1: Spontaneous pneumothorax
done 2: Cluster headache
done 3: Boerhaave
done 4: Spontaneous rib fracture
done 5: GERD
done 6: HIV (initial infection)
done 7: Anemia
done 8: Viral pharyngitis
done 9: Inguinal hernia
done 10: Myasthenia gravis
done 11: Whooping cough
done 12: Anaphylaxis
done 13: Epiglottitis
done 14: Guillain-Barré syndrome
done 15: Acute laryngitis
done 16: Croup
done 17: PSVT
done 18: Atrial fibrillation
done 19: Bronchiectasis
done 20: Allergic sinusitis
done 21: Chagas
done 22: Scombroid food poisoning
done 23: Myocarditis
done 24: Larygospasm
done 25: Acute dystonic reactions
done 26: Localized edema
done 27: SLE
done 28: Tuberculosis
done 29: Unstable angina
done 30: Stable angina
done 31: Ebola
done 32: Acute otitis media
done 33: Panic attack
done 34: Bronchospasm / acute asthma exacerbation
done 35: Bronchitis
done 36: Acute COPD exacerbation / infection
done 37: Pulmonary embolism
done 38: URTI
done 39: Influenza
done 40: Pneumonia
done 41: Acute rhinosinu

In [51]:
diagnosis_df_valid["predicted_diagnosis"] = diagnosis_df_valid[disease_list].progress_apply(lambda x: pred(x), axis=1)

100%|██████████| 132448/132448 [09:00<00:00, 244.84it/s]


In [52]:
diagnosis_df_valid["is_matched"] = diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis"]].progress_apply(lambda x: validate(x), axis=1)

100%|██████████| 132448/132448 [00:02<00:00, 46416.80it/s]


In [53]:
diagnosis_df_valid["hit_rate"] = hit_rate

In [54]:
diagnosis_df_valid.is_matched.value_counts()

True     107691
False     24757
Name: is_matched, dtype: int64

In [55]:
acc = diagnosis_df_valid.is_matched.value_counts().to_dict()
acc["error_rate"] = acc[False]/len(diagnosis_df_valid)
acc

{True: 107691, False: 24757, 'error_rate': 0.18691863976805992}

In [56]:
np.mean(hit_rate)

0.9840657983621638

In [57]:
acc["mean_hit_rate"] = np.mean(hit_rate)

In [58]:
# hit rate of misdiagnosed
hit_rate_misdiagnosed = diagnosis_df_valid[diagnosis_df_valid["is_matched"]==False]["hit_rate"]
hit_rate_misdiagnosed.plot.hist()
plt.ylabel("Frequency")
plt.title("Hit rate Distribution - Misdiagnosed")
plt.savefig(f'{base_path}\\output\\questionnaire\\hit_rate_misdiagnosed_dt.jpg', bbox_inches='tight')
plt.clf()

<Figure size 640x480 with 0 Axes>

In [59]:
np.mean(hit_rate_misdiagnosed)

0.9372862797823415

In [60]:
acc["mean_hit_rate_misdiagnosed"] = np.mean(hit_rate_misdiagnosed)

In [61]:
with open(f"{base_path}\\output\\error_analysis_questionnaire\\validation_metric_dt_questionnaire.json", "w") as outfile: 
    json.dump(acc, outfile, indent=True)

In [62]:
diagnosis_df_valid[["PATHOLOGY", "predicted_diagnosis", "is_matched", "hit_rate"] + disease_list].to_csv(f"{base_path}\\output\\error_analysis_questionnaire\\validation_dt_df_all_patients_questionnaire.csv")